In [1]:
# !pip install python-scalpel


In [2]:
from scalpel.import_graph.import_graph import ImportGraph, Tree

In [14]:
import os
print("Current working directory:", os.getcwd())
target_dir = os.getcwd()+'/data_request_api'  # Use the current working directory as the target directory
print("Target directory:", target_dir)
import_graph = ImportGraph(target_dir)
import_graph.build_dir_tree()



Current working directory: /Users/daniel.ellis/WIPwork/CMIP7_DReq_Software
Target directory: /Users/daniel.ellis/WIPwork/CMIP7_DReq_Software/data_request_api


In [15]:
all_leaf_nodes = import_graph.get_leaf_nodes()

module_imports = {}
for node in all_leaf_nodes:
    module_dict = import_graph.parse_import(node.ast)
    module_imports[import_graph.leaf2root(node)] = module_dict

module_imports

{'data_request_api.version': {'typing': ['Tuple', 'Union']},
 'data_request_api.data_request_api.__init__.py': {'version': ['*']},
 'data_request_api.data_request_api.tests.test_dreq_content': {'data_request_api.content': ['dreq_content'],
  'data_request_api.utilities.logger': ['change_log_file',
   'change_log_level']},
 'data_request_api.data_request_api.tests.test_config_cli': {'pathlib': ['Path']},
 'data_request_api.data_request_api.tests.test_optimize': {'__future__': ['print_function',
   'division',
   'unicode_literals',
   'absolute_import'],
  'data_request_api.query.data_request': ['DataRequest'],
  'data_request_api.content.dreq_content': ['_dreq_res'],
  'data_request_api.utilities.tools': ['read_json_input_file_content'],
  'data_request_api.content.dump_transformation': ['correct_dictionaries',
   'transform_content_inner',
   'get_transformed_content',
   'get_transform_settings'],
  'data_request_api.tests': ['filepath']},
 'data_request_api.data_request_api.tests.te

In [ ]:
import json
from graphviz import Digraph
import os

# Your import data
import_data = module_imports  # replace with your dict

# Setup dot path
dot_path = "/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/bin/"
os.environ["PATH"] = dot_path + os.pathsep + os.environ["PATH"]

dot = Digraph(comment="CMIPLD Module Import Graph", engine="sfdp")
dot.attr(rankdir='TB')
dot.attr(compound='true', splines='true', overlap='false', concentrate='true')

def categorize_node(name):
    if name.endswith(".py"):
        return "pyfile"
    if name.startswith("cmipld."):
        level = name.count('.')
        if level == 1:
            return "top-level"
        else:
            return "nested"
    else:
        return "external"

all_nodes = set()
target_nodes = set()

for module, imports in import_data.items():
    all_nodes.add(module)
    for imp_module in imports.keys():
        if isinstance(imp_module, int):
            continue
        all_nodes.add(imp_module)
        target_nodes.add(imp_module)

styles = {
    "external": {"color": "blue", "shape": "box", "style": "filled", "fillcolor": "#c6dbef"},
    "nested": {"color": "darkgreen", "shape": "ellipse", "style": "filled", "fillcolor": "#a1d99b"},
    "top-level": {"color": "red", "shape": "diamond", "style": "filled", "fillcolor": "#fcbba1"},
    "isolated": {"color": "grey", "shape": "circle", "style": "filled", "fillcolor": "#eeeeee"},
    "pyfile": {"color": "orange", "shape": "rhombus", "style": "filled", "fillcolor": "#fdd0a2"},
}

for node in all_nodes:
    if not node:
        continue
    category = categorize_node(node)
    if node not in target_nodes and node not in import_data:
        category = "isolated"
    style = styles.get(category, {})
    dot.node(node,
             color=style.get("color"),
             shape=style.get("shape"),
             style=style.get("style"),
             fillcolor=style.get("fillcolor"))

for module, imports in import_data.items():
    for imp_module, items in imports.items():
        if isinstance(imp_module, int):
            continue
        try:
            dot.edge(module, imp_module)
        except Exception as e:
            print(f"Error adding edge from {module} to {imp_module}: {e}")

with open("import_graph.dot", "w") as f:
    f.write(dot.source)

dot.render("import_graph", format="pdf", cleanup=True)
print("Graph written to import_graph.dot and rendered to import_graph.pdf")


Graph written to import_graph.dot and rendered to import_graph.pdf
